In [ ]:
# the goal is using multiGPUs
# put the nn model on GPU
model.gpu()
# copy tesnor on GPU(we should assign a new tensor and use it on GPU)
mytensor = my_tensor.gpu()

# import libaries and data
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

# Parameters and DataLoaders
input_size = 5
output_size = 2

batch_size = 30 # size of minibatch?
data_size = 100 # num of samples

# make a dummy(random) dataset 
class RandomDataset(Dataset):
    
    def __init__(self,size,length):
        self.len = length
        self.data = torch.randn(length,size)
        
    def __getitem__(self,index):
        return self.data[index]
    
    def __len__(self):
        return self.len
    
rand_loader = DataLoader(dataset = RandomDataset(input_size,100),
                        batch_size=batch_size,shuffle=True)
# what's shuffle? Data shuffled at each epoch 
        
# we use DataParallel on any model(CNN,RNN,Capsule net and ...)
# here we only try a linear model 
class Model(nn.Module):
    def __init__(self,input_size,output_size):
        super(model,self).__init__()
        self.fc = nn.Linear(input_size,output_size)
        
    def forward(self,input):
        output = self.fc(input)
        print("  In Model: input size", input.size(),
              "output size", output.size())

        return output
 
# create Model and Dataparallel
# First we need to make model for testing whether we have multiple 
# GPUs. If we have multiGPUs, then we can wrap our model using nn.DataParallel
# then we can put our model on GPU by model.gpu()

model = Model(input_size, output_size)
if torch.cuda.device_count()>1: # check num of GPUs
    print("Let's use", torch.cuda.device_count(),"GPUs!")
    # dim = 0 [30,xxx] -> [10,...], [10,...] [10,...] on GPUs
    model = nn.DataParallel(model) # use parallel structure 
    
if torch.cuda.is_available():
   model.cuda()

# run the model
for data in rand_loader:
    if torch.cuda.is_available():
        input_var = Variable(data.cuda())
    else:
        input_var = Variable(data)

    output = model(input_var)
    print("Outside: input size", input_var.size(),
          "output_size", output.size())

# if we batch 30 inputs and 30 outputs, the results expected are also 
# 30 inputs and 30 outputs, but with 2 GPUs, results are splited to 2 parts: 15,15  

# DataParallel splits your data automatically and sends job orders to multiple models on several GPUs. 
# After each model finishes their job, DataParallel collects and merges the results before returning it to you.